In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn import preprocessing 
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from __future__ import division

In [3]:
train = pd.read_table("data_content", header=None, skip_blank_lines=False)
label = pd.read_table("label", header=None, dtype='category', skip_blank_lines=False)

In [4]:
train.columns = ['question']
label.columns = ['group']

In [5]:
print train.shape
print label.shape

(67213, 1)
(67213, 1)


In [6]:
ratio = 0.4
# Separate each muscle group data
chest = train[label['group']=='Chest']
chest_label = label[label['group']=='Chest']
chest_len = int(chest.shape[0]*ratio)

shoulder = train[label['group']=='Shoulders']
shoulder_len = int(shoulder.shape[0]*ratio)
shoulder_label = label[label['group']=='Shoulders']

back = train[label['group']=='Back']
back_label = label[label['group']=='Back']
back_len = int(back.shape[0]*ratio)

leg = train[label['group']=='Leg']
leg_label = label[label['group']=='Leg']
leg_len = int(leg.shape[0]*ratio)

tricep = train[label['group']=='Triceps']
tricep_label = label[label['group']=='Triceps']
tricep_len = int(tricep.shape[0]*ratio)

bicep = train[label['group']=='Biceps']
bicep_label = label[label['group']=='Biceps']
bicep_len = int(bicep.shape[0]*ratio)

ab = train[label['group']=='Abs']
ab_label = label[label['group']=='Abs']
ab_len = int(ab.shape[0]*ratio)

glute = train[label['group']=='Glutes']
glute_label = label[label['group']=='Glutes']
glute_len = int(glute.shape[0]*ratio)

In [7]:
# Take only the top ratio% of each group of data
train_data = pd.concat([chest[0:chest_len], shoulder[0:shoulder_len], back[0:back_len], 
                        leg[0:leg_len], tricep[0:tricep_len], bicep[0:bicep_len], ab[0:ab_len], glute[0: glute_len]])
print train_data.shape
train_label = pd.concat([chest_label[0:chest_len], shoulder_label[0:shoulder_len], back_label[0:back_len], 
                   leg_label[0:leg_len], tricep_label[0:tricep_len], bicep_label[0:bicep_len], ab_label[0:ab_len],
                         glute_label[0: glute_len]])
print train_label.shape

(26883, 1)
(26883, 1)


In [8]:
y_label = pd.Series(train_label['group'],dtype='category')
y_label.cat.categories 
y_label.cat.categories = [0,1,2,3,4,5,6,7]

In [21]:
#label binizer
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2,3,4,5,6,7])
label = lb.transform(y_label)

In [28]:
#initialize TFIDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3),stop_words="english")
data = vectorizer.fit_transform(train_data['question'])

In [29]:
# train test split
train_data, testX, train_label, testY = train_test_split(data, label, test_size=0.2, random_state=0)

ValueError: Found input variables with inconsistent numbers of samples: [21506, 26883]

In [24]:
# tensorflow setup
numFeatures = train_data.shape[1]
numLabels = 8
numEpochs = 100
learningRate = tf.train.exponential_decay(learning_rate=0.08,
                                          global_step= 1,
                                          decay_steps=train_data.shape[0],
                                          decay_rate= 0.9,
                                          staircase=True)


X = tf.placeholder(tf.float32, [None, numFeatures])
Y = tf.placeholder(tf.float32, [None, numLabels])

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=(np.sqrt(6/numFeatures+
                                                         numLabels+1)),
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=(np.sqrt(6/numFeatures+numLabels+1)),
                                    name="bias"))

In [26]:
# tensorflow operation
init = tf.global_variables_initializer()

apply_weights_OP = tf.matmul(X, weights, name="apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 
activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

cost_OP = tf.nn.l2_loss(activation_OP-Y, name="squared_error_cost")
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

In [27]:
# tensorflow session
sess = tf.Session()
sess.run(init)

correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(Y,1))
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

cost = 0
diff = 1

# Training epochs
for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        # Run training step
        step = sess.run(training_OP, feed_dict={X: train_data, Y: train_label})
        # Report occasional stats
        if i % 10 == 0:
            # Add epoch to epoch_values
            epoch_values.append(i)
            # Generate accuracy stats on test data
            summary_results, train_accuracy, newCost = sess.run(
                [accuracy_OP, cost_OP], 
                feed_dict={X: train_data, Y: train_label}
            )
            # Add accuracy to live graphing variable
            accuracy_values.append(train_accuracy)
            # Add cost to live graphing variable
            cost_values.append(newCost)
            # Write summary stats to writer
            writer.add_summary(summary_results, i)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g"%(i, train_accuracy))
            print("step %d, cost %g"%(i, newCost))
            print("step %d, change in cost %g"%(i, diff))

            

# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={X: testX, 
                                                                Y: testY})))

ValueError: could not convert string to float: The rear delts play an important role in physique development, strength and stabilization of the shoulder. See how you can use Lying Rolling Floor Laterals to get the work done without the damage. Che